# Project - PPTX summarizer AI Assistant


In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display, update_display

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()



OpenAI API Key exists and begins sk-proj-


In [3]:

import os
from pptx import Presentation
import openai  # adapt if you use a different client

class PptxSummary:
    def __init__(self, name: str):
        self.name = name

    def summary(self):
        # Load an existing PowerPoint
        prs = Presentation(self.name)
        newtext = ""

        # print(prs)
        # Loop over all slides
        for i, slide in enumerate(prs.slides, start=1):
            # print(f"\n--- Slide {i} ---")
            newtext += f"\n\n--- Slide {i} ---"
            
            # Loop over shapes (text boxes, titles, placeholders, etc.)
            for shape in slide.shapes:
                if shape.has_text_frame:  # Only shapes that can contain text
                    for paragraph in shape.text_frame.paragraphs:
                        # Collect text from each run in the paragraph
                        text = "".join(run.text for run in paragraph.runs)
                        # print(text)
                        newtext+= "\n"
                        newtext += text
        # print(newtext)
        return newtext

        
system_message = "You are a helpful assistant for a company and you can summarize the given topic based on the given pptx name. "
system_message += "Give short, courteous answers, no more than 10 sentence. "
system_message += "Always be accurate. If the presentation .pptx file does not exist, say so. Respond in markdown."

def user_message(path):
    ppt = PptxSummary(path)
    summarization_message = ppt.summary()
    message = "You need to summarize the a pptx file."
    message += f"The context of that pptx file is here: {summarization_message}"
    message += "Give the concise information in small paragraphs. "
    return message

def pptx_summary(path):
    if os.path.exists(path):
        result = "The file does not exist"
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message(path)}
              ],
        )
        result = response.choices[0].message.content
        return result
    else:
        return "The file does not exist"


    

# print("Current working directory:", os.getcwd())
# print("Files here:", os.listdir())


In [4]:
print(pptx_summary("Marie_Curie.pptx"))

# Summary of "Marie Curie: Highlights of a Notable Life"

The presentation details the life and achievements of Marie Curie, born in Warsaw, Poland in 1867. She pursued her studies in physics and mathematics in Paris, setting the stage for her groundbreaking scientific career.

Marie Curie made history as the first woman to win a Nobel Prize and is renowned for her discovery of radioactivity, alongside her husband, Pierre Curie. Her noteworthy accomplishments include winning Nobel Prizes in both Physics and Chemistry, highlighting her significant contributions to science.

Curie's legacy extends beyond her scientific achievements; she shattered barriers for women in science and her research ultimately aided in the development of medical X-rays. She is celebrated as one of the most influential scientists of all time, leaving an enduring impact on both science and society.


# My tools

In [5]:
print(pptx_summary("bts.pptx"))

The file does not exist


## Tools


In [6]:
# There's a particular dictionary structure that's required to describe our function:

summary_function = {
    "name": "pptx_summary",
    "description": "Get the summary for the given pptx file, you need to call this function, if user asks for a pptx file, if it is outside of the pptx file, tell that you do not know.",
    "parameters": {
        "type": "object",
        "properties": {
            "path": {
                "type": "string",
                "description": "The name of the presentation",
            },
        },
        "required": ["path"],
        "additionalProperties": False
    }
}

In [7]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": summary_function}]

## Getting OpenAI to use our Tool



In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(f"message: {message}")
        response, city = handle_tool_call(message)
        print(f"city: {city}, response: {response}")
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        print(f"response: {response}")
            
    return response.choices[0].message.content

In [9]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(f"tool_call: {tool_call}")
    arguments = json.loads(tool_call.function.arguments)
    print(f"arguments: {arguments}")
    path = arguments.get('path')
    print(f"path: {path}")
    summary = pptx_summary(path)
    # print(f"price: {price}")
    response = {
        "role": "tool",
        "content": json.dumps({"path": path,"summary": summary}),
        "tool_call_id": tool_call.id
    }
    print(f"response: {response}")
    return response, path

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_qgCGcnIRiknlEaT6eGFlPyN4', function=Function(arguments='{"path":"nelson_mandela.pptx"}', name='pptx_summary'), type='function')])
tool_call: ChatCompletionMessageFunctionToolCall(id='call_qgCGcnIRiknlEaT6eGFlPyN4', function=Function(arguments='{"path":"nelson_mandela.pptx"}', name='pptx_summary'), type='function')
arguments: {'path': 'nelson_mandela.pptx'}
path: nelson_mandela.pptx
response: {'role': 'tool', 'content': '{"path": "nelson_mandela.pptx", "summary": "# Summary of \\"Nelson Mandela: Highlights of a Notable Life\\" \\n\\nThis presentation covers the life and impact of Nelson Mandela. Born in 1918 in South Africa, Mandela became politically active in the fight against apartheid after studying law. \\n\\nHe emerged as a prominent leader in the anti-apartheid movement but faced imprisonment for 27